# What this notebook does

The overall goal is produce a table that can be used to create a feature-set ready for model training. To do that, we fist pull commit information about a github project, then format that info as a table where each row is a separate commit and columns hold info about the commit. Here is the target, where each row represents a single commit:

<img src="https://www.dropbox.com/s/c2wrow809najo9s/Screenshot%202019-07-15%2011.11.08.png?raw=1">

After this we will build a second table that compacts infomation into days. This will result in a table that looks like this:

<img src="https://www.dropbox.com/s/53ddnk7201mmecz/Screenshot%202019-07-15%2011.09.43.png?raw=1">

## Assumptions of this notebook

1. The code and notebooks we will be using, including this one, are in the following folder: <code>/ideas-uo/machine_learning/predicting_project_activity</code>.   

2. Two tables will be written to <code>'/ideas-uo/machine_learning/predicting_project_activity'</code>: (1) <code>project +  '_commits_table.csv'</code> and (2) <code>project +  '_days_table.csv'</code>.

3. This notebook should be started in folder <code>/ideas-uo/machine_learning/predicting_project_activity</code>.


In [1]:
import os
starting_dir = os.getcwd() #~/ideas-uo/machine_learning/predicting_project_activity
starting_dir

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/machine_learning/predicting_project_activity'

In [2]:
#move up to uo-ideas folder - must be better way
%cd ../../

/Users/fickas/Dropbox/boyana/my_work/ideas-uo


In [3]:
repository_dir = os.getcwd() #~/ideas-uo/
save_dir = repository_dir + '/machine_learning/predicting_project_activity/'

In [4]:
code_dir = repository_dir + '/code'


In [5]:
import sys
sys.path.append(code_dir)

### At some point figure out why search path screwed up

Everything works fine from command line. But need these 2 get search path set up in jupyter.

In [6]:
sys.path.append('/Users/fickas/anaconda/envs/py3/lib/python3.3/site-packages/pymongo/')
sys.path.append('/Users/fickas/anaconda/envs/py3/lib/python3.3/site-packages/')

## Parameters for this notebook

In [8]:
project_name = 'petsc'  #or latte, etc.
project_url = 'https://github.com/petsc/petsc.git'  #'https://github.com/spack/spack.git'

# Part 1

Our goal is to load in and parse project data and produce a pandas dataframe that is commit-based, i.e., every row represents a separate commit.

## Our library

We wrote the hpcl library to act as a github puller. See /ideas-uo/code/hpcl/ for source.

In [9]:
import os, sys, subprocess
from gitutils import command
from gitutils import gitcommand
from gitutils import remotecommand
#import mysql.connector    #works from command line but not here


In [10]:
commander = RemoteCommand.RemoteCommand()  #you will be prompted for github username and password

Enter GitHub credentials
Username: fickas
Password: ········


In [11]:
#move back to code dir /ideas-uo/code

os.chdir(code_dir)


## Load repo off github

Pulling the project info can take some time.

In [12]:
#Produces nested Python lists and dictionaries

def checkoutSubrepos(repos,tdir):
    #currdir = os.getcwd()
    for repopath in repos.keys():
        if repopath == tdir: continue
        retcode, out, err = command.Command('git checkout %s%s' % repos[repopath]).run(dryrun=dry_run)
    return


if __name__ == "__main__":
    
    print('Going to load repos...')

    #Load the repo to clone from the following file
    #project_info = open(os.path.join('.',project_url),'r').readlines()

    #Move to the tmp directory that will hold the repo
    currdir = os.getcwd()
    tmpdir = os.path.join(currdir,'tmp')  #sff: fails if does not already exist
    #os.chdir(tmpdir)

    commander = GitCommand.GitCommand(tmpdir)

    #Download repo
    print('Cloning: ' + project_name)
    commander.cloneRepo(project_url)

    print('Finished cloning repo.')

    #Setup DB connection, this will probably change soon.
    #mydb = mysql.connector.connect(host="localhost", port="3307", user="pythondb", passwd="********", database="gitstats")
    #print(mydb)
    #mycursor = mydb.cursor()
    #sql = "INSERT INTO stats (reponame, stats) VALUES(%s, %s)"

    #Now run commands on the repo

    if not os.path.isdir(project_name): print(('missing repo', name))

    print('Checking: ' + project_name)
    prefix,versions = commander.getRepoVersions(project_name)
    print(prefix)
    print(versions)

    #Gets all the commit data in a dictionary format {authorname:{'total_commits':0, 'commits':{'date':'', 'message':'', 'diffs':{'filename':'', 'diff':''}}}}
    active_developers = commander.getRepoCommitData(project_name)

    print(('number of developers', len(active_developers)))

    """ stats = getStats('.',repo)
    ts = repo + ', ' + out.strip().split('-')[0] 
    buf = ts
    for i in range(0,len(category_names)): 
        buf += ', %d' % stats[category_names[i]]
        linecounts[i].append(stats[category_names[i]]*0.001)
    outfile.write(buf+'\n')
    #years.append(int(ts))
    print(stats)

    val = (repo, buf)"""
    #mycursor.execute(sql,val)
    #mydb.commit()






Going to load repos...
Cloning: petsc
Finished cloning repo.
Checking: petsc

[b'5f4ed19a8efdb486a315195012d444ef692ac3ef', b'e6f63ad2ab5728109effa7fc4268fc5f39596e69', b'9628acf9e69feaf5b20a332c1abeb272c6569839', b'2b6d935b2ebe034503326b4dee965ac49e70a83a', b'42b35daeee85a2865211138058e46364a55fba67', b'6b41c64db32b5d27bf1ffdc0ebdce35eda45500a', b'624d586ae11407e421e8be9da896d71e46a73fcc', b'd1bca9b51d479d7f87cfb3f2949e1def8bb0a48a', b'7f5ff6fd825ec37dd17ca520ff753cbee35ff4f1', b'106f7b34c4de9115c040144f51174a77a7dafc70', b'99667eae6d03a65cbb1ac6c97c1eaaa333aaedbd', b'6024bd2c273c6b4ac03c2bd49d1968cdcd0f9df3', b'54be42616974234e5fbfa50cb10e1315fe5d2421', b'1a7bf5d4b73a1242167f49f9b7133387521e485c', b'aa85bbbfd89bbd07a357c477fa76d6d7944350fe', b'dbd978078a9f5e9133273eeb046fa1244b394e07', b'f74f53555c8c46e222f524114de7085640faea86', b'd60bcec1f80cd32b662c0d186375959f7cad5719', b'ee1d3c3675fda8bdc596b90041c524cb92173008', b'735aa83eb89a6a5bbd80b0b19421900e3a8f1d3b', b'32ec92c24b3fabe9006

## Let's see what we have

In [13]:
dev_tups = list(active_developers.items())  #get it in form easier to see
dev_tups[0]

(b'Barry Smith <bsmith@mcs.anl.gov>',
 {'commits': [{'date': b'Thu Aug 1 09:54:14 2019 -0500',
    'diffs': [{'diff': ['-   MatMPIBAIJSetPreallocationCSR - Allocates memory for a sparse parallel matrix in BAIJ format',
       '-   (the default parallel PETSc format).',
       '+   MatMPIBAIJSetPreallocationCSR - Creates a sparse parallel matrix in BAIJ format using the given nonzero structure and (optional) numerical values'],
      'filename': 'a/src/mat/impls/baij/mpi/mpibaij.c b/src/mat/impls/baij/mpi/mpibaij.c'}],
    'id': b'664954b6e79d13912fafd6f958f3b260df0a822f',
    'message': '    Fix manual pages for MatXXXYBAIJSetPreallocationCSR() routines\n    \n    Commit-type: documentation\n'},
   {'date': b'Tue Jul 30 19:54:56 2019 -0500',
    'diffs': [{'diff': ['+/*@C',
       '+    PetscIntCast - casts a PetscInt64 (which is 64 bits in size) to a PetscInt (which may be 32 bits in size), generates an',
       '+         error if the PetscInt is not large enough to hold the number.'

## Nested structure is complicated

Code below dives into the structure for illustration. It can be skipped on production run.

## Goal reminder

Goal is to reformat the data. Currently the key is user and the value is a nested structure that includes all the commits for that person. We want to invert this. We'd like a list of commits, where each commit has info including developer name.

Note Python is a mess when it comes to time zones. What is supposed to work, <code>%z</code>, does not. See discussion here: https://stackoverflow.com/questions/3305413/python-strptime-and-timezones.

In particular, these fail when include UTC: <code>'%a %b %m %X %y %z'</code> and <code>'%c %z'</code>.

We are resorting to a utility package.

In [14]:
import datetime
from dateutil import parser

## Helper function

Given a list of diff strings (see list_of_diff_strings above), do a count of lines of code changed for each string. At moment this is trivial given we assume each string represents a single change (+ or -). So in theory we could just take the length of change_list as locc. However, we want to verify that each string starts with a + or - and print a warning if not.
<p>
Eventually we want to get back to looking at replacing a line with another one. This will be counted as 2 changes, a + and a -, when it should only be counted as 1. On our to-do list is to further process change_list to catch these type of replace actions.

In [15]:
import re  #use in future

In [16]:
#This is a trivial method until we start thinking about replace actions
def count_changes(change_list):
    change_count = 0
    #Go through each change represented as a string, e.g., '+ share/spack/qa/.*'
    for change in change_list:
        #Big assumption: single + or - per change and it is the first char
        if change[0] in '+-':
            change_count += 1
        else:
            print(('missing +- at index 0', change))
    #TODO: worry about a replace being counted as 2 instead of 1
    return change_count
        

## Create a list of dicts

Each dict will end up as a row in a commit-based table.

<code>entry</code> below is the 2-tuple version of a single developer record. It has 2 items, the developer name and the complicated stuff.

In [17]:
all_commits = []

for entry in dev_tups:
    name = entry[0]                               #'Gilles Fourestey <gilles.fourestey@epfl.ch>'
    commit_list = entry[1]['commits']      #a list of dictionaries with date, diffs, message as keys
    seen = []                               #need this because getting duplicates
    for a_commit in commit_list:
        if a_commit in seen: continue
        if not a_commit['diffs']: continue  #skip commit with empty diff
        seen.append(a_commit)
        new_dict = {}
        date = parser.parse(a_commit['date'])         #"Tue Aug 2 14:58:31 2016 +0200"
        new_dict['year'] = date.year
        new_dict['doy'] = min(365, int(date.strftime('%j'))) #punting on leap year
        new_dict['month'] = date.month
        new_dict['day_of_month'] = date.day
        new_dict['day_name'] = date.strftime('%A')  #e.g., Tuesday
        new_dict['utc_offset'] = date.utcoffset().seconds  #e.g., 7200

        new_dict['name'] = name
        new_dict['message'] = a_commit['message']
        
        diffs = a_commit['diffs']  #list of dictionaries diff, filename
        
        filenames = []
        changes_total = 0
        for real_diff in diffs:   #a dictionary diff filename
            filenames.append(real_diff['filename'])
            the_diffs = real_diff['diff']  #list of strings
            if the_diffs:
                change_count = count_changes(the_diffs)
                changes_total += change_count
            else:
                 print(('warning: empty diff list for', name, date.month, date.day, a_commit['message'], a_commit['id']))
        new_dict['filenames'] = filenames
        new_dict['locc'] = changes_total
        all_commits.append(new_dict)
        
    

In [18]:
print(('total commits', len(all_commits)))

('total commits', 60265)


In [19]:
import pandas as pd

In [20]:
raw_table = pd.DataFrame(all_commits)  #gotta love pandas

In [21]:
#reorder way columns print
commits_table = raw_table[['month', 'day_of_month', 'day_name', 'year',
                           'locc', 'message', 'filenames', 'name', 'doy', 'utc_offset']]

In [22]:
#df_sorted = df_commits.sort_values(by='date')  #fails because of UTC
commits_table = commits_table.sort_values(['year', 'doy'])  #primary and secondary
commits_table = commits_table.reset_index(drop=True)

# Part 2

Now produce a day-based table using the commits table as a starting point.

In [23]:
import datetime
from dateutil import parser
from datetime import timedelta


## Here is starting date

starting_year = commits_table.loc[0,'year']
starting_month = commits_table.loc[0,'month']
starting_day = commits_table.loc[0,'day_of_month']
starting_obj = datetime.date(starting_year, starting_month, starting_day)


In [24]:
## Here is ending date

ending_year = commits_table.iloc[-1]['year']
ending_month = commits_table.iloc[-1]['month']
ending_day = commits_table.iloc[-1]['day_of_month']
ending_obj = datetime.date(ending_year, ending_month, ending_day)


In [25]:
#We should end up with a list of this length, i.e., a list item for each day.

td = ending_obj - starting_obj


## Approach

Goal: for everyday between starting and ending dates, create a row for that day.

Actual method: loop through rows of commit table. Keep values needed to (a) count rows with same date, (b) count days skipped leading to a sequence of 0 entries, and (c) determine when switch years so can reset values.

Note that things like messages and filenames will be combined into a list that goes with a day, i.e., can have mulitple filenames and multiple messages associated with the activity in one day.

In [26]:
#Fill out this new table. These columns will be the fodder for features in a feature set.

days_table = pd.DataFrame(columns= ['month', 'day_of_month', 'day_of_week', 'year', 'total_commits',
                           'total_loccs', 'total_messages', 'total_filenames', 'total_names',
                                    'doy', 'issues'])


In [27]:
current_day = int(commits_table.loc[0,'doy'])  #day of year: 1-365 (or 366 on leap years)
current_year = commits_table.loc[0,'year']
dnint = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}

names = []
messages = []
filenames = []
total_commits = 0
total_loccs = 0


for i in range(len(commits_table)):

    #pull out date pieces
    year = int(commits_table.loc[i,'year'])
    day_of_year = int(commits_table.loc[i,'doy'])
    
    #check if change years, e.g., change from 2013 to 2014
    if year!=current_year:
        current_year = year
        diff = day_of_year + (365 - current_day)  #account for skipped days at end of old year
    else:
        diff = day_of_year - current_day
    
    #diff now holds number of days incremented
    
    #No diff so same day - increment all tracked values for the day
    if diff==0:
        total_loccs += commits_table.loc[i,'locc']
        total_commits += 1
        messages.append(commits_table.loc[i,'message'])
        names.append(commits_table.loc[i,'name'])
        filenames.append(commits_table.loc[i,'filenames'])
        continue
    
    #Now things get interesting. We need to move back in time to beginning edge of gap. If gap is size diff,
    #then move back diff days. That will give us the date before the gap begins.
    
    #First build date object - easier to do arithmetic on. This is date on ending edge of gap.
    month = int(commits_table.loc[i,'month'])
    day_of_month = int(commits_table.loc[i,'day_of_month'])
    end_gap_date = datetime.datetime(year, month, day_of_month)   #current row we are looking at
    
    begin_gap_date = end_gap_date - datetime.timedelta(days=diff) #looking back in time
    
    #record row values for begin gap date
    prior_day_name = dnint[begin_gap_date.strftime('%A')]  #convert to int 1-7
    prior_month = begin_gap_date.month
    prior_day_of_month = begin_gap_date.day
    prior_year = begin_gap_date.year
    prior_doy = min(365, int(begin_gap_date.strftime('%j')))

    #build row and append
    new_row = {'day_of_week': prior_day_name, 'month': prior_month, 'day_of_month': prior_day_of_month,
               'doy': prior_doy, 'year': prior_year,
               'total_messages': messages, 'total_names': names, 'total_filenames': filenames,
               'total_loccs': total_loccs, 'total_commits': total_commits}
    days_table = days_table.append(new_row, ignore_index=True)
    
    #Whew. Took care of recording data for the beginning data of gap.
    
    #diff = 1 so tomorrow is here :) Just reset things since no dates skipped
    if diff == 1:
        total_loccs = commits_table.loc[i,'locc']
        total_commits = 1
        messages = [commits_table.loc[i,'message']]
        names = [commits_table.loc[i,'name']]
        filenames = [commits_table.loc[i,'filenames']]
        current_day = day_of_year
        continue
    
    #we have a gap! need to fill in with 0 feature values for each day in gap
    if diff > 1:
        date_obj = begin_gap_date
        total_loccs = 0
        total_commits = 0
        messages = []
        names = []
        filenames = []
        for j in range(diff-1):
            date_obj += datetime.timedelta(days=1)  #handles month change overs
            day_name = dnint[date_obj.strftime('%A')]
            new_row = {'day_of_week': day_name, 'month': date_obj.month, 'day_of_month': date_obj.day,
               'doy': min(365, int(date_obj.strftime('%j'))), #punting on leap year
                'year': date_obj.year, 'total_names': names, 'total_filenames': filenames,
               'total_messages': messages, 'total_loccs': total_loccs, 'total_commits': total_commits}
            days_table = days_table.append(new_row, ignore_index=True)
        #record the new commit we just saw for data at end of gap
        total_loccs = commits_table.loc[i,'locc']
        total_commits = 1
        messages = [commits_table.loc[i,'message']]
        names = [commits_table.loc[i,'name']]
        filenames = [commits_table.loc[i,'filenames']]
        current_day = day_of_year  #now on new date
        continue
    
    print((i, day_of_year, year, diff))
    raise Exception  #should never get here

#check if have any values accumulated.
if total_commits>0:
    final_commit_row = commits_table.iloc[-1]
    end_date = datetime.datetime(final_commit_row['year'], final_commit_row['month'], final_commit_row['day_of_month'])
    new_row = {'day_of_week': dnint[end_date.strftime('%A')], 'month': end_date.month, 'day_of_month': end_date.day,
               'doy': final_commit_row['doy'], 'year': end_date.year,
               'total_messages': messages, 'total_names': names, 'total_filenames': filenames,
               'total_loccs': total_loccs, 'total_commits': total_commits}
    days_table = days_table.append(new_row, ignore_index=True) 

In [28]:
x = days_table.tail(30)
x

,month,day_of_month,day_of_week,year,total_commits,total_loccs,total_messages,total_filenames,total_names,doy,issues
9044,7,3,3,2019,7,64,[ Fix --download-colpack\n \n Cleanup...,[[a/config/BuildSystem/config/package.py b/con...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Vaclav...",184,NaN
9045,7,4,4,2019,9,42,[ Several GNU package installers were initi...,[[a/config/BuildSystem/config/packages/ADOLC.p...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Barry ...",185,NaN
9046,7,5,5,2019,10,49,[ Remove additional uneeded values from pet...,[[a/config/BuildSystem/config/compilers.py b/c...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Barry ...",186,NaN
9047,7,6,6,2019,4,10,[ Remove outdated packages variable that go...,[[a/config/BuildSystem/config/packages/pnetcdf...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Barry ...",187,NaN
9048,7,7,7,2019,5,11,[ Filter Mat_ name that can be different on...,[[a/src/mat/examples/tests/ex230.c b/src/mat/e...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Matthe...",188,NaN
9049,7,8,1,2019,11,48,[ config.package.GNUPackage packages must i...,[[a/config/BuildSystem/config/packages/hdf5.py...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Barry ...",189,NaN
9050,7,9,2,2019,16,39,[ Introduce HAVE_PGF90_COMPILER to replace ...,[[a/config/BuildSystem/config/compilers.py b/c...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Barry ...",190,NaN
9051,7,10,3,2019,10,238,[ GNUPackages did not init with GNUPackage;...,[[a/config/BuildSystem/config/packages/hdf5.py...,"[b'Barry Smith <bsmith@mcs.anl.gov>', b'Barry ...",191,NaN
9052,7,11,4,2019,14,731,[ Plex ex36: Must cast real numbers for out...,[[a/src/dm/impls/plex/examples/tests/ex36.c b/...,"[b'Matthew G. Knepley <knepley@gmail.com>', b'...",192,NaN
9053,7,12,5,2019,9,47,"[ Plex ex36: Fixed type for array access\n,...",[[a/src/dm/impls/plex/examples/tests/ex36.c b/...,"[b'Matthew G. Knepley <knepley@gmail.com>', b'...",193,NaN


## Now add in issue data

In [29]:
commander = RemoteCommand.RemoteCommand()  #you will be prompted for github username and password

Enter GitHub credentials
Username: fickas
Password: ········


In [30]:
reponame = project_name+'/'+project_name

issues = commander.getIssues(reponame)  #reponame is the full project name. e.g. spack/spack

Checking: petsc/petsc
Link accessing:  https://api.github.com/repos/petsc/petsc/issues?state=all&page=1
Request successful......  <Response [200]>
Request length......  5
Link accessing:  https://api.github.com/repos/petsc/petsc/issues?state=all&page=2
RESULTS 5


In [46]:
len(issues) #12199

5

In [33]:
iss0 = issues[0]  #get the first issue in the long list
list(iss0.keys())

['url',
 'repository_url',
 'labels_url',
 'comments_url',
 'events_url',
 'html_url',
 'id',
 'node_id',
 'number',
 'title',
 'user',
 'labels',
 'state',
 'locked',
 'assignee',
 'assignees',
 'milestone',
 'comments',
 'created_at',
 'updated_at',
 'closed_at',
 'author_association',
 'body']

In [34]:
iss0

{'assignee': None,
 'assignees': [],
 'author_association': 'NONE',
 'body': 'I want to install PETSc with compex numbers. My configurations are like this:\r\n`export PETSC_DIR=/home/afrah/petsc-master\r\nexport PETSC_ARCH=arch-linux2-c-debug-complex\r\n\r\n./configure PETSC_DIR=/home/afrah/petsc-master PETSC_ARCH=arch-linux2-c-debug-complex  prefix=/home/afrah/petsc-master/source-complex \\\r\n--with-cc=mpicc --with-cxx=mpicxx --with-fc=mpif90 --download-superlu=1 \\\r\n--download-superlu_dist=1 --download-metis=1 --download-parmetis=1 \\\r\n --download-blacs --download-scalapack=1 --download-mumps=1  \\\r\n--with-valgrind=1  --with-valgrind-dir=[/usr/local/bin] \\\r\n--with-complex=1`\r\n\r\nbut after checking , I got the following results:\r\n\r\n===============================================================================                                                                                          Compilers:                                                             

In [35]:
foo = parser.parse(iss0['updated_at'])

In [36]:
foo.year

2018

In [37]:
foo.strftime('%j')

'174'

In [38]:
rows = days_table.loc[(days_table['year'] == foo.year) & (days_table['doy'] == int(foo.strftime('%j')))]

In [39]:
rows.index[0]

8669

In [40]:
days_table['issues'] = [[] for i in range(len(days_table))]

In [41]:
missing_days = []
for issue in issues:
    created = parser.parse(issue['created_at'])
    updated = parser.parse(issue['updated_at']) if issue['updated_at'] else issue['updated_at']
    closed = parser.parse(issue['closed_at']) if issue['closed_at'] else issue['closed_at']
    row = days_table.loc[(days_table['year'] == created.year) & (days_table['doy'] == int(created.strftime('%j')))]
    if not len(row):
        missing_days.append((issue, created))
        continue
    i = row.index[0]
    current_list = days_table.loc[i, 'issues']
    current_list.append(issue['body'])
    days_table.loc[i, 'issues'] = current_list
    

In [42]:
len(missing_days)

0

In [43]:
days_table.head()

,month,day_of_month,day_of_week,year,total_commits,total_loccs,total_messages,total_filenames,total_names,doy,issues
0,9,22,4,1994,1,14,[ Initial revision\n],"[[a/maint/mkpdir b/maint/mkpdir, a/maint/torcs...",[b'Barry Smith <bsmith@mcs.anl.gov>'],265,[]
1,9,23,5,1994,0,0,[],[],[],266,[]
2,9,24,6,1994,0,0,[],[],[],267,[]
3,9,25,7,1994,0,0,[],[],[],268,[]
4,9,26,1,1994,0,0,[],[],[],269,[]


In [44]:
days_table.tail()

,month,day_of_month,day_of_week,year,total_commits,total_loccs,total_messages,total_filenames,total_names,doy,issues
9069,7,28,7,2019,3,174,[ tests: add alt files for ksp_ksp_tutorial...,[[a/src/ksp/ksp/examples/tutorials/output/ex74...,"[b'Satish Balay <balay@mcs.anl.gov>', b'Pierre...",209,[]
9070,7,29,1,2019,7,88,[ Missing code around something that needs ...,[[a/src/docs/website/documentation/installatio...,"[b'BarryFSmith <bsmith@mcs.anl.gov>', b'BarryF...",210,[]
9071,7,30,2,2019,2,50,[ Add PetscIntCast() to check for integer o...,"[[a/include/petscsys.h b/include/petscsys.h], ...","[b'Barry Smith <bsmith@mcs.anl.gov>', b'Vaclav...",211,[]
9072,7,31,3,2019,7,50,[ fix compile warnings on arch-linux-pkgs-6...,"[[a/include/petscsys.h b/include/petscsys.h], ...","[b'Satish Balay <balay@mcs.anl.gov>', b'BarryF...",212,[]
9073,8,1,4,2019,1,3,[ Fix manual pages for MatXXXYBAIJSetPreall...,[[a/src/mat/impls/baij/mpi/mpibaij.c b/src/mat...,[b'Barry Smith <bsmith@mcs.anl.gov>'],213,[]


In [48]:
row = days_table.loc[2357]

In [49]:
len(row['issues'])

18

In [50]:
ilist = [len(days_table.loc[i,'issues']) for i in range(len(days_table)) if len(days_table.loc[i,'issues'])>1]
len(ilist)

1236

In [51]:
max(ilist)

42

In [ ]:
fee_fie_foe()

## Write to file

In [45]:
os.chdir(save_dir)
filename = project_name+'_feature_set_table_alt_2.csv'
days_table.to_csv(filename, index=False)